<a href="https://colab.research.google.com/github/HYChangTAMU/Financial-Data-Analysis/blob/main/Assignment5/Project5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Date 11/29/2023

#-------------------------------------------------------------------------------

!pip install mat73
!pip install ZCAPM
import mat73
import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
import json
import datetime
import pytz
import ZCAPM
from statsmodels.stats.weightstats import ttest_ind, ttost_paired
from sklearn.metrics import r2_score
from google.colab import files


#-------------------------------------------------------------------------------

a_factors = pd.read_csv('/content/drive/MyDrive/Assignment5/mu_sigma_vwt_2022.csv')
# a_factors.rename({'VWRETD-Rf': 'R_a.R_f', 'VWStd': 'sigma_a'}, axis=1, inplace=True)
a_factors.rename({'DATE': 'Date'}, axis=1, inplace=True)

#-------------------------------------------------------------------------------

f6 = pd.read_csv('/content/drive/MyDrive/Assignment4/data/6_factors_daily.csv')
f6.drop(range(14979, 15123), inplace = True)


#-------------------------------------------------------------------------------

daily_return = pd.DataFrame(mat73.loadmat('/content/drive/MyDrive/Assignment4/data/stock_daily_return.mat')['stock_ret'])
daily_return[0] = daily_return[0].astype('int64')
daily_return.rename({0: 'Date'}, axis=1, inplace=True)
daily_return.drop(range(879), inplace = True)
daily_return.reset_index(drop = True, inplace = True)


#-------------------------------------------------------------------------------

# start and end of 714 months
# month_start_end records the dates
# month_start_end_index records the indices

month_start_end = []
dates = f6['Date'][0:-1].tolist()
for i, d in enumerate(dates):
  if d%100 < dates[i-1]%100:
    month_start_end.append(dates[i-1])
    month_start_end.append(d)
month_start_end.append(month_start_end[0])
month_start_end = month_start_end[1:]

mse_index = []
for i in range(len(month_start_end)//2):
  mse_index.append(f6['Date'].index[f6['Date'] == month_start_end[2*i]].tolist())
  mse_index.append(f6['Date'].index[f6['Date'] == month_start_end[2*i + 1]].tolist())

month_start_end_index = []
for i in range(len(month_start_end)):
  month_start_end_index.append(mse_index[i][0])
month_start_end[-1] = 20221230
month_start_end_index[-1] = 14978

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.8 MB/s eta 0:00:00


In [14]:
dates_conv = f6[['Date']]
Date = pd.to_datetime(dates_conv['Date'], format="%Y%m%d").dt.date
YearMonth = dates_conv['Date'].apply(lambda x: x//100)
YearMonth.rename('YearMonth', inplace = True)

0        196307
1        196307
2        196307
3        196307
4        196307
          ...  
14974    202212
14975    202212
14976    202212
14977    202212
14978    202212
Name: YearMonth, Length: 14979, dtype: int64

In [20]:
def window_data(n):
  # n_min = 0
  # n_max = 702
  fac_data = f6[month_start_end_index[2*n]:month_start_end_index[2*n+23]+1]
  # fac_data.drop(columns = 'Date', inplace = True)

  ret_data = daily_return[month_start_end_index[2*n]:month_start_end_index[2*n+23]+1]
  ret_data.dropna(how = 'any', axis = 1, inplace = True)
  ret_data.columns = [str(x) for x in ret_data.columns]
  ret_data.drop(columns = 'Date', inplace = True)
  ret_data = ret_data.sub(fac_data['RF'], axis = 0)

  fac_data.drop(columns = 'RF', inplace = True)

  return ret_data.join(fac_data, how = 'inner')

In [22]:
window_data(1)

<ipython-input-20-844d8115a3f5>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ret_data.dropna(how = 'any', axis = 1, inplace = True)
<ipython-input-20-844d8115a3f5>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ret_data.drop(columns = 'Date', inplace = True)
<ipython-input-20-844d8115a3f5>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fac_data.drop(columns = 'RF', inplace = True)


,7,15,31,58,66,103,138,146,154,162,...,20306,22186,30615,Date,Mkt-RF,SMB,HML,RMW,CMA,Mom
22,-0.005595,-0.011000,-0.018282,-0.028094,-0.006261,-0.001544,-0.001172,-0.005962,-0.025815,-0.014460,...,-0.023987,-0.011000,-0.011000,19630801,-0.08,-0.12,-0.13,0.13,-0.08,-0.03
23,-0.000247,-0.011000,-0.008555,-0.006652,-0.015717,0.000710,0.006032,-0.003481,-0.018519,-0.011000,...,-0.037316,-0.002453,-0.025286,19630802,0.29,-0.14,0.16,-0.05,0.03,0.11
24,-0.005681,0.026037,-0.011000,0.014974,-0.011000,-0.006370,-0.006215,-0.001050,0.004152,-0.011000,...,0.029541,0.005949,-0.011000,19630805,0.57,-0.42,0.16,0.01,0.08,0.17
25,0.018101,-0.046714,0.013390,-0.011000,-0.006261,-0.001783,-0.018143,-0.009522,-0.018463,-0.007528,...,-0.023987,-0.011000,0.003493,19630806,0.60,-0.36,0.25,-0.02,-0.04,0.24
26,-0.005859,-0.011000,-0.011000,-0.002561,-0.006283,-0.013283,-0.013398,-0.003556,0.019075,-0.014460,...,-0.011000,-0.019333,-0.025286,19630807,-0.19,0.01,0.18,-0.16,0.14,0.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,0.005676,0.087000,-0.015141,-0.020018,0.000889,-0.024494,-0.013000,-0.006072,-0.007048,-0.013000,...,-0.027286,-0.005908,-0.032802,19640727,-0.48,-0.06,0.28,-0.01,0.17,0.01
270,0.000333,-0.058455,-0.017292,-0.009466,-0.017566,-0.013000,0.005519,-0.022174,-0.024834,-0.018650,...,0.001493,-0.027085,-0.023101,19640728,-0.24,0.14,0.05,0.20,0.04,0.02
271,0.010026,0.034619,-0.015155,-0.011239,-0.008413,-0.015326,-0.017545,-0.017630,-0.001024,-0.007318,...,-0.027286,0.001286,0.007408,19640729,0.13,-0.10,-0.03,0.20,-0.06,-0.16
272,-0.009785,-0.013000,0.002119,-0.013000,-0.013000,-0.017662,-0.017566,-0.019977,-0.007083,-0.004525,...,-0.027493,-0.027085,-0.003000,19640730,0.13,-0.14,-0.03,-0.30,0.08,-0.09


## Testing the ZCAPM Package


In [4]:
test = ZCAPM.Testing()

In [48]:
test.sorted_portfolio_daily_excess_return.dtypes

YearMonth       int64
SMALL.LoBM    float64
ME1.BM2       float64
ME1.BM3       float64
ME1.BM4       float64
SMALL.HiBM    float64
ME2.BM1       float64
ME2.BM2       float64
ME2.BM3       float64
ME2.BM4       float64
ME2.BM5       float64
ME3.BM1       float64
ME3.BM2       float64
ME3.BM3       float64
ME3.BM4       float64
ME3.BM5       float64
ME4.BM1       float64
ME4.BM2       float64
ME4.BM3       float64
ME4.BM4       float64
ME4.BM5       float64
BIG.LoBM      float64
ME5.BM2       float64
ME5.BM3       float64
ME5.BM4       float64
BIG.HiBM      float64
dtype: object

In [ ]:
#Set Parameters for rolling estimations and EM Algorithm. For information on parameters see doc string of _EM_loop method
tol = .001
MaxIter = 1000
criterion = 1
width = 12

######## IMPORTANT########
#trims off all of the first "width" amount of monthly returns to ensure that the Fama-MacBeth test is performed
#OUT OF SAMPLE
monthly_excess_return = test.sorted_portfolio_monthly_excess_return.iloc[width:,:]
portfolio_excess_return = test.sorted_portfolio_daily_excess_return
factor_return = test.factor_daily_return

YearMonth = monthly_excess_return.index

#create pandas series for mkt ret, mkt sigma, and factors. Convert indices of these series and portfolio return dataframe
#to be the YearMonth list. Useful for indexing purposes while testing
mu = (factor_return.loc[:,"R_a.R_f"])
sigma = (factor_return.loc[:,"sigma_a"])
facs_ret = factor_return.loc[:,['YearMonth','R_a.R_f','SMB','HML']]

mu.index = factor_return.YearMonth
sigma.index = factor_return.YearMonth
portfolio_excess_return.set_index('YearMonth',inplace = True)
facs_ret.set_index('YearMonth',inplace =True)

In [ ]:
#calculates time series factor loadings for each portfolio. See Testing class for information on the arguments of each
#method
zcapm_results = test.estZCAPM(portfolio_excess_return,mu,sigma,tol,MaxIter,criterion,width)
# ff3_results = test.estLinearModel(portfolio_excess_return,facs_ret,width)
# capm_results = test.estLinearModel(portfolio_excess_return,facs_ret.loc[:,['R_a.R_f']],width)

#adjust the zeta estimates for each portfolio to monthly estimates
zeta_cols = zcapm_results.columns[zcapm_results.columns.str.contains('zeta')]
zcapm_results.loc[:,zeta_cols] = zcapm_results.loc[:,zeta_cols]*21

In [ ]:
zcapm_results

,SMALL.LoBM beta,SMALL.LoBM zeta,ME1.BM2 beta,ME1.BM2 zeta,ME1.BM3 beta,ME1.BM3 zeta,ME1.BM4 beta,ME1.BM4 zeta,SMALL.HiBM beta,SMALL.HiBM zeta,...,BIG.LoBM beta,BIG.LoBM zeta,ME5.BM2 beta,ME5.BM2 zeta,ME5.BM3 beta,ME5.BM3 zeta,ME5.BM4 beta,ME5.BM4 zeta,BIG.HiBM beta,BIG.HiBM zeta
YearMonth,,,,,,,,,,,,,,,,,,,,,
196412,1.099749,-0.352402,1.002181,-0.210200,0.771063,0.500876,0.710273,0.004014,0.945831,0.499077,...,1.124476,-0.107168,0.834803,-0.160272,0.955905,0.184712,1.272193,-0.094054,1.350527,-0.155759
196501,1.060569,-0.113919,0.955749,-0.080973,0.747277,0.600714,0.691036,0.231385,0.929258,0.764179,...,1.122048,-0.059299,0.821589,-0.368722,0.988572,0.161355,1.262883,-0.110518,1.334198,-0.089832
196502,1.048046,-0.107963,0.933864,-0.078404,0.787325,0.621845,0.704563,0.419167,0.914045,0.819605,...,1.108315,-0.071948,0.834434,-0.347596,1.028872,0.156084,1.237076,-0.294047,1.344365,-0.381886
196503,1.034353,0.084690,0.917893,0.226941,0.822781,0.656015,0.717946,0.426505,0.910280,1.064458,...,1.108643,-0.014010,0.825842,-0.395529,1.038206,-0.045976,1.233576,-0.372574,1.361912,-0.091889
196504,0.986344,0.360841,0.914810,0.167825,0.835412,0.948003,0.707024,0.642613,0.884527,1.328441,...,1.129973,0.038474,0.817556,-0.568598,1.018836,-0.169676,1.219187,-0.431916,1.355104,-0.338629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201507,1.101494,-0.243116,1.033481,0.200763,0.973565,-0.572815,0.893294,-0.275749,0.786956,-0.707984,...,0.979922,0.459627,0.985791,0.087685,1.019801,-0.308904,0.998823,-0.746996,1.021590,-0.291420
201508,1.014193,-0.354925,0.968642,-0.042300,0.895075,-0.502718,0.820303,-0.191385,0.733233,-0.599921,...,1.005292,0.435372,0.996916,-0.044439,1.021598,-0.224758,0.999322,-0.635467,1.042385,-0.298744
201509,0.976435,-0.288037,0.939126,-0.214933,0.858127,-0.385946,0.781477,0.032363,0.713461,-0.636765,...,0.995680,0.364788,0.996467,-0.107497,1.029551,-0.169864,0.994847,-0.586142,1.056515,-0.431432


In [ ]:
zeta_cols

Index(['SMALL.LoBM zeta', 'ME1.BM2 zeta', 'ME1.BM3 zeta', 'ME1.BM4 zeta',
       'SMALL.HiBM zeta', 'ME2.BM1 zeta', 'ME2.BM2 zeta', 'ME2.BM3 zeta',
       'ME2.BM4 zeta', 'ME2.BM5 zeta', 'ME3.BM1 zeta', 'ME3.BM2 zeta',
       'ME3.BM3 zeta', 'ME3.BM4 zeta', 'ME3.BM5 zeta', 'ME4.BM1 zeta',
       'ME4.BM2 zeta', 'ME4.BM3 zeta', 'ME4.BM4 zeta', 'ME4.BM5 zeta',
       'BIG.LoBM zeta', 'ME5.BM2 zeta', 'ME5.BM3 zeta', 'ME5.BM4 zeta',
       'BIG.HiBM zeta'],
      dtype='object')

In [ ]:
zcapm_results.loc[:,zeta_cols]

,SMALL.LoBM zeta,ME1.BM2 zeta,ME1.BM3 zeta,ME1.BM4 zeta,SMALL.HiBM zeta,ME2.BM1 zeta,ME2.BM2 zeta,ME2.BM3 zeta,ME2.BM4 zeta,ME2.BM5 zeta,...,ME4.BM1 zeta,ME4.BM2 zeta,ME4.BM3 zeta,ME4.BM4 zeta,ME4.BM5 zeta,BIG.LoBM zeta,ME5.BM2 zeta,ME5.BM3 zeta,ME5.BM4 zeta,BIG.HiBM zeta
YearMonth,,,,,,,,,,,,,,,,,,,,,
196412,-0.352402,-0.210200,0.500876,0.004014,0.499077,-0.605331,-0.658862,0.684994,0.157181,0.503195,...,-0.156183,0.620258,0.340749,0.618030,-0.063821,-0.107168,-0.160272,0.184712,-0.094054,-0.155759
196501,-0.113919,-0.080973,0.600714,0.231385,0.764179,-0.077648,-0.314648,0.738219,0.402202,0.547969,...,0.173334,0.732824,0.560017,0.646730,0.041204,-0.059299,-0.368722,0.161355,-0.110518,-0.089832
196502,-0.107963,-0.078404,0.621845,0.419167,0.819605,0.053663,-0.031228,0.838520,0.536398,0.501555,...,0.146123,0.918986,0.668347,0.503446,0.028441,-0.071948,-0.347596,0.156084,-0.294047,-0.381886
196503,0.084690,0.226941,0.656015,0.426505,1.064458,0.174025,-0.185600,0.814180,0.408188,0.372989,...,0.138168,0.826827,0.423701,0.308734,0.355287,-0.014010,-0.395529,-0.045976,-0.372574,-0.091889
196504,0.360841,0.167825,0.948003,0.642613,1.328441,0.091722,0.159890,0.975288,0.631682,0.504741,...,0.393915,0.833792,0.444226,0.631056,0.517906,0.038474,-0.568598,-0.169676,-0.431916,-0.338629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201507,-0.243116,0.200763,-0.572815,-0.275749,-0.707984,0.370058,0.101120,-0.276387,-0.190859,-1.526293,...,0.487809,0.117967,0.036948,-0.337233,-0.822159,0.459627,0.087685,-0.308904,-0.746996,-0.291420
201508,-0.354925,-0.042300,-0.502718,-0.191385,-0.599921,0.056807,0.258155,-0.067565,-0.243410,-1.486250,...,0.352635,0.162528,0.080906,-0.154906,-0.778399,0.435372,-0.044439,-0.224758,-0.635467,-0.298744
201509,-0.288037,-0.214933,-0.385946,0.032363,-0.636765,0.018677,0.129148,-0.058712,0.001092,-1.148411,...,0.290824,0.217093,0.164061,-0.011268,-0.640843,0.364788,-0.107497,-0.169864,-0.586142,-0.431432


In [ ]:
#Runs the Fama-MacBeth Test for each portfolio
ZCAPM = test.FamaMacBeth(monthly_excess_return,zcapm_results,['beta','zeta'],'ZCAPM')
FF3 = test.FamaMacBeth(monthly_excess_return,ff3_results,['R_a.R_f','SMB','HML'],'Fama-French 3 Factor')
CAPM = test.FamaMacBeth(monthly_excess_return,capm_results,['R_a.R_f',],'CAPM')

In [ ]:
ZCAPM

,coefficients,t-values
ZCAPM,,
intercept,0.7593262880472017,3.0875681939793758
beta,-0.1783317961459172,-0.7265999715044813
zeta,0.4885823411522058,4.299951332644644
Single Regression Approach R-squared,0.9690609647879035,


In [ ]:
FF3

,coefficients,t-values
Fama-French 3 Factor,,
intercept,0.8899926349195396,4.625249967909042
R_a.R_f,-0.3742676286214684,-1.7778283203036696
SMB,0.18226260438326758,1.3752332078248828
HML,0.3025136631944703,2.5440381412417037
Single Regression Approach R-squared,0.6470600399973208,


In [ ]:
CAPM

,coefficients,t-values
CAPM,,
intercept,0.9215175378932576,3.738539529018011
R_a.R_f,-0.2986757946856537,-1.1877174962510997
Single Regression Approach R-squared,0.5198257918234428,
